<a href="https://www.kaggle.com/code/kiranpatil7022/seoul-bike-insights-and-predictions?scriptVersionId=147381511" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/seoul-bike-trip-duration-prediction/For_modeling.csv


In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import  train_test_split

In [3]:
df = pd.read_csv(
    "/kaggle/input/seoul-bike-trip-duration-prediction/For_modeling.csv",
    dtype={
        'Duration' : 'int8',
        'Distance' : 'int8',
        'PLong' : 'float32',
        'PLatd' : 'float32',
        'DLong' : 'float32',
        'DLatd' : 'float32',
        'Haversine' : 'float32',
        'Pmonth' : 'int8',
        'Pday' : 'int8',
        'Phour' : 'int8',
        'Pmin' : 'int8',
        'PDweek' : 'int8',
        'Dmonth' : 'int8',
        'Dday' : 'int8',
        'Dhour' : 'int8',
        'Dmin' : 'int8',
        'DDweek' : 'int8',
        'Temp' : 'float32',
        'Precip' : 'float32',
        'Wind' : 'float32',
        'Humid' : 'float32',
        'Solar' : 'float32',
        'Snow' : 'float32',
        'GroundTemp' : 'float32',
        'Dust' : 'float32'
    },
    index_col=0
).sample(frac=1)
df.head()

,Duration,Distance,PLong,PLatd,DLong,DLatd,Haversine,Pmonth,Pday,Phour,...,Dmin,DDweek,Temp,Precip,Wind,Humid,Solar,Snow,GroundTemp,Dust
8462325,29,122,37.566845,126.896446,37.571476,126.889687,0.787422,11,2,16,...,22,4,18.799999,0.0,1.4,32.0,1.09,0.0,18.100000,47.0
4861449,14,74,37.571102,127.023560,37.586899,127.020752,1.773893,8,9,22,...,46,3,27.500000,0.0,0.4,82.0,0.00,0.0,26.200001,28.0
6589360,57,48,37.526264,126.920509,37.526264,126.920509,0.000000,9,22,20,...,23,5,20.400000,0.0,1.2,52.0,0.00,0.0,17.000000,35.0
2045776,10,-106,37.554951,126.910835,37.557499,126.923805,1.177895,5,22,12,...,53,1,22.900000,0.0,1.1,51.0,1.80,0.0,34.500000,61.0
2788945,71,42,37.555054,126.937569,37.547249,126.939293,0.881085,6,11,7,...,59,0,19.100000,0.0,1.1,59.0,0.38,0.0,21.700001,10.0


In [4]:
df.shape

(9601139, 25)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9601139 entries, 8462325 to 7343378
Data columns (total 25 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Duration    int8   
 1   Distance    int8   
 2   PLong       float32
 3   PLatd       float32
 4   DLong       float32
 5   DLatd       float32
 6   Haversine   float32
 7   Pmonth      int8   
 8   Pday        int8   
 9   Phour       int8   
 10  Pmin        int8   
 11  PDweek      int8   
 12  Dmonth      int8   
 13  Dday        int8   
 14  Dhour       int8   
 15  Dmin        int8   
 16  DDweek      int8   
 17  Temp        float32
 18  Precip      float32
 19  Wind        float32
 20  Humid       float32
 21  Solar       float32
 22  Snow        float32
 23  GroundTemp  float32
 24  Dust        float32
dtypes: float32(13), int8(12)
memory usage: 659.3 MB
